# 1. Import Dependencies

In [1]:
# Import Gym Stuff
import gym
from gym import Env # New stuff
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete # New stuff

# Import helpers
import numpy as np
import random
import os

# Import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

C:\Users\iDLaber\miniconda3\envs\baselines\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Types of Spaces 

In [2]:
Discrete(3).sample()

2

In [3]:
Box(0,1, shape=(3,3)).sample() # 3*3 space, but the value between 0 and 1

array([[0.9232501 , 0.37715292, 0.1928447 ],
       [0.7978541 , 0.59528553, 0.33386797],
       [0.66028327, 0.49496475, 0.39551896]], dtype=float32)

In [4]:
Tuple((Discrete(3), Box(0,1,shape=(3,)))).sample()

(2, array([0.37693185, 0.35651517, 0.15531203], dtype=float32))

In [5]:
Dict({'height':Discrete(2), "speed":Box(0,100,shape=(1,)), 'color': MultiBinary(3)}).sample()

OrderedDict([('color', array([0, 0, 1], dtype=int8)),
             ('height', 1),
             ('speed', array([52.42343], dtype=float32))])

In [6]:
MultiBinary(4).sample()

array([1, 0, 1, 1], dtype=int8)

In [7]:
MultiDiscrete([5,2,2]).sample() # lower 5, lower 2, lower 2 sampleing

array([1, 0, 1], dtype=int64)

# 3. Building an Environment
- Build an agent to give us the best shower possible
- Randomly temperature
- 37 and 39 degrees

In [8]:
Box(low = np.array([0]), high=np.array([100]),shape=(1,)).sample()

C:\Users\iDLaber\miniconda3\envs\baselines\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


array([25.827604], dtype=float32)

In [9]:
val = Discrete(3).sample()
print(val, val-1)

1 0


In [10]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low = np.array([0]), high=np.array([100]), shape=(1,))
        self.state = 38+random.randint(-3,3) # Initial State - 38 degree +- 3 -> goal: 37~39
        self.shower_length = 60 
        
    def step(self, action):
        # shower length is decreased by 1 second every step
        
        # Apply temperature adj
        self.state += action-1
        
        # Decrease shower time
        self.shower_length -= 1
        
        # Calculate Reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        return self.state, reward, done, info
        
    def render(self):
        # No render
        pass
    
    def reset(self):
        # Reset initial temperature and shower time
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state
    


In [11]:
env = ShowerEnv()

In [12]:
env.action_space.sample()

0

In [13]:
env.observation_space.sample()

array([11.314268], dtype=float32)

# 4. Test Environment

In [14]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
# env.close()

Episode:1 Score:-56
Episode:2 Score:-8
Episode:3 Score:-12
Episode:4 Score:-30
Episode:5 Score:-24


# 5. Train Model

In [15]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy',env, verbose=1, tensorboard_log = log_path)
# Tabular data, Tensor-based data -> use MlpPolicy

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [16]:
model.learn(total_timesteps = 40000)

Logging to Training\Logs\PPO_15
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -28.4    |
| time/              |          |
|    fps             | 1670     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -31.6       |
| time/                   |             |
|    fps                  | 1161        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011184023 |
|    clip_fraction        | 0.108       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.00133     

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -24.4       |
| time/                   |             |
|    fps                  | 941         |
|    iterations           | 11          |
|    time_elapsed         | 23          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.012266371 |
|    clip_fraction        | 0.03        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 9.24e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 34.4        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00358    |
|    value_loss           | 75.4        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

# 6. Save Model

In [18]:
shower_path = os.path.join('Training','Saved Models','Shower_Model_PPO')

In [19]:
model.save(shower_path)

In [25]:
evaluate_policy(model, env, n_eval_episodes=10)

(-24.0, 54.99090833947008)